In [2]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator
import model_definitions

### Open input file

In [2]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3
DISCARD=5

In [3]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
df=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))
    #dfs.append(df[i])

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [6]:
united.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5310813,5.5585,4.1559,0.70930,0.18634,0.13008,-0.028081,-1.381588,1.713180,8,16,0.280890,-0.021667,0.22,167,10
5310814,5.6789,4.1709,0.41373,-0.06862,0.41472,0.071998,-2.260208,-0.257397,8,16,0.318749,0.006139,0.22,167,10
5310815,5.6559,4.0909,-1.09178,-0.23122,-0.69446,-0.768151,-13.461109,3.683377,8,16,0.356455,0.033412,0.22,167,10
5310816,4.9543,4.3687,-4.54340,2.16567,-11.03359,0.065644,-7.969290,4.537201,8,16,0.389484,0.062536,0.22,167,10
5310817,3.9566,4.6490,-3.28609,-0.29356,-3.04057,1.133997,23.822023,-24.215317,8,16,0.415861,0.093529,0.22,167,10


In [7]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [8]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.00,0,0
1,-0.01641,0.46479,-0.787199,3.679083,-0.6641,-0.555253,-37.836440,0.133991,4,32,-0.124219,0.395531,0.22,0,0
2,-0.23364,1.15674,-0.505374,2.481521,-5.8806,0.090642,23.921020,5.117202,4,32,-0.127334,0.410954,0.22,0,0
3,-0.29309,1.54297,0.437547,1.485258,-2.4037,0.808292,22.235918,7.911897,4,32,-0.131242,0.431527,0.22,0,0
4,-0.23889,1.90650,-0.478706,2.162672,1.1955,-0.339512,-11.018061,5.533980,4,32,-0.134427,0.456947,0.22,0,0


In [17]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 2654974 races: 168 n batches: 2641534 / 512
Length: 1327474 races: 168 n batches: 1314034 / 512
Length: 1328370 races: 168 n batches: 1314930 / 512


In [11]:
x_y=gen_test[0]
x=x_y[0][0]
print(x)
x_y[1][0]

tf.Tensor(
[[-9.511800e-01 -4.491993e-01 -8.937000e-01  1.900000e+01  3.000000e+01
   1.314111e-01  1.860025e-01]
 [-8.038800e-01 -1.858376e-01 -1.003000e-01  1.900000e+01  3.000000e+01
   5.041341e-02  1.878775e-01]
 [-3.850800e-01  8.231067e-01  2.371900e+00  1.900000e+01  3.000000e+01
  -3.241699e-02  1.914452e-01]
 [-7.251200e-01  6.701610e-01  8.360000e-01  1.900000e+01  3.000000e+01
  -1.167692e-01  2.002864e-01]
 [-6.196400e-01 -3.903000e-02 -3.292000e-01  1.900000e+01  3.000000e+01
  -2.032064e-01  2.110611e-01]
 [-7.321000e-01 -1.515370e-01 -5.748000e-01  1.900000e+01  3.100000e+01
   2.084766e-01  2.207746e-01]
 [-7.026500e-01 -5.427800e-02 -4.730000e-02  1.900000e+01  3.100000e+01
   1.180599e-01  2.297460e-01]
 [-5.758500e-01  2.494030e-01  2.121000e-01  1.900000e+01  3.100000e+01
   2.564941e-02  2.387954e-01]
 [ 3.353000e-01  2.071953e+00  5.460300e+00  1.900000e+01  3.100000e+01
  -6.675618e-02  2.551040e-01]
 [-4.672100e-01  1.320263e+00  1.216700e+00  1.900000e+01  3.1

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([4.27571, 0.06039, 8.468  ])>

### Model definition

#### Train and print

In [18]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [11]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=10,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

Epoch 1/10
5109/5109 [==============================] - 488s 95ms/step - loss: 8.0866 - mean_absolute_error: 1.8331 - accuracy: 0.0000e+00 - val_loss: 6.2619 - val_mean_absolute_error: 1.5699 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5109/5109 [==============================] - 489s 96ms/step - loss: 5.8418 - mean_absolute_error: 1.5124 - accuracy: 0.0000e+00 - val_loss: 6.1840 - val_mean_absolute_error: 1.5562 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5109/5109 [==============================] - 487s 95ms/step - loss: 5.8164 - mean_absolute_error: 1.5084 - accuracy: 0.0000e+00 - val_loss: 6.1767 - val_mean_absolute_error: 1.5555 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5108/5109 [============================>.] - ETA: 0s - loss: 5.8146 - mean_absolute_error: 1.5083 - accuracy: 1.2746e-07
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
5109/5109 [==============================] - 490s 96ms/step - loss: 5.8151 - mean_ab

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [20]:
lr =1e-4
units=32
cells=3
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{name}')

Epoch 1/100
5109/5109 [==============================] - 506s 98ms/step - loss: 6.1255 - mean_absolute_error: 1.6144 - accuracy: 0.0000e+00 - val_loss: 5.9518 - val_mean_absolute_error: 1.5549 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
5109/5109 [==============================] - 497s 97ms/step - loss: 4.4482 - mean_absolute_error: 1.3534 - accuracy: 1.2743e-07 - val_loss: 6.2036 - val_mean_absolute_error: 1.5591 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/100
5108/5109 [============================>.] - ETA: 0s - loss: 4.0611 - mean_absolute_error: 1.2801 - accuracy: 0.0000e+00
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
5109/5109 [==============================] - 494s 97ms/step - loss: 4.0614 - mean_absolute_error: 1.2801 - accuracy: 0.0000e+00 - val_loss: 6.5179 - val_mean_absolute_error: 1.5938 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/100
5109/5109 [==============================] - 493s 96ms/step - loss: 3.9721 - mea

In [27]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}_PosAssoluta')

INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/LSTM2_32_1e-05_NormFalse_PosAssoluta\assets


In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,30,5],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)